In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = "NF-UNSW-NB15-v3.parquet"
data = pd.read_parquet(file_name)

In [3]:
data.Label.value_counts()

Label
0    2151027
1      91904
Name: count, dtype: int64

In [4]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [5]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [6]:
data.Attack.unique()

array(['Benign', 'Fuzzers', 'Exploits', 'Backdoor', 'Generic', 'DoS',
       'Reconnaissance', 'Shellcode', 'Analysis', 'Worms'], dtype=object)

In [7]:
data = data.groupby(by='Attack').sample(frac=0.1, random_state=13)

In [8]:
data.groupby(by="Attack").count()

,FLOW_START_MILLISECONDS,FLOW_END_MILLISECONDS,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,FTP_COMMAND_RET_CODE,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Analysis,123,123,123,123,123,123,123,123,123,123,...,123,123,123,123,123,123,123,123,123,123
Backdoor,345,345,345,345,345,345,345,345,345,345,...,345,345,345,345,345,345,345,345,345,345
Benign,215103,215103,215103,215103,215103,215103,215103,215103,215103,215103,...,215103,215103,215103,215103,215103,215103,215103,215103,215103,215103
DoS,505,505,505,505,505,505,505,505,505,505,...,505,505,505,505,505,505,505,505,505,505
Exploits,3882,3882,3882,3882,3882,3882,3882,3882,3882,3882,...,3882,3882,3882,3882,3882,3882,3882,3882,3882,3882
Fuzzers,2559,2559,2559,2559,2559,2559,2559,2559,2559,2559,...,2559,2559,2559,2559,2559,2559,2559,2559,2559,2559
Generic,476,476,476,476,476,476,476,476,476,476,...,476,476,476,476,476,476,476,476,476,476
Reconnaissance,1129,1129,1129,1129,1129,1129,1129,1129,1129,1129,...,1129,1129,1129,1129,1129,1129,1129,1129,1129,1129
Shellcode,159,159,159,159,159,159,159,159,159,159,...,159,159,159,159,159,159,159,159,159,159


In [ ]:
X = data.drop(columns=["Attack", "Label", "FLOW_START_MILLISECONDS", "FLOW_END_MILLISECONDS",
                       "SRC_TO_DST_IAT_MIN", "SRC_TO_DST_IAT_MAX", "SRC_TO_DST_IAT_AVG",
                       "SRC_TO_DST_IAT_STDDEV", "DST_TO_SRC_IAT_MIN", "DST_TO_SRC_IAT_MAX",
                       "DST_TO_SRC_IAT_AVG", "DST_TO_SRC_IAT_STDDEV"])
y = data[["Attack", "Label"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)

In [10]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [11]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [12]:
# (Modified)
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()
X_train['id'] = X_train.index

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()
X_test['id'] = X_test.index

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [13]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h,id
287406,59.166.0.8,149.171.126.4,4.732152e-09,0.000000e+00,0.003486,0.000005,0.000317,0.000004,1.505900e-09,1.611550e-09,...,0.001493,0.002850,2.127056e-08,2.097309e-08,4.396601e-09,4.397286e-09,0.0,4.240589e-09,"[4.732152333835413e-09, 0.0, 0.003485589444774...",287406
610356,59.166.0.9,149.171.126.6,6.932495e-10,7.976222e-09,0.000215,0.000002,0.000128,0.000002,9.621527e-10,9.621527e-10,...,0.000000,0.000000,4.010061e-09,4.009943e-09,1.969528e-08,1.969835e-08,0.0,1.899640e-08,"[6.932494894241552e-10, 7.97622160421992e-09, ...",610356
1456518,59.166.0.2,149.171.126.2,6.381741e-07,2.401223e-07,0.060366,0.000354,0.038930,0.000379,2.030844e-07,2.173323e-07,...,0.183020,0.183020,4.321202e-07,4.840666e-07,5.929219e-07,5.930143e-07,0.0,5.718822e-07,"[6.381740567526722e-07, 2.40122348914616e-07, ...",1456518
2204716,59.166.0.0,149.171.126.0,1.390953e-08,2.334703e-08,0.000405,0.000006,0.000476,0.000004,4.426392e-09,4.736937e-09,...,0.002792,0.003191,4.531849e-09,4.766241e-09,1.292322e-08,1.292524e-08,0.0,1.246465e-08,"[1.3909530687793818e-08, 2.3347026113255993e-0...",2204716
1090799,59.166.0.1,149.171.126.3,4.602189e-09,0.000000e+00,0.000213,0.000003,0.001485,0.000003,1.464542e-09,1.567291e-09,...,0.001980,0.001320,8.530939e-11,8.530939e-11,4.275853e-09,4.276519e-09,0.0,4.124125e-09,"[4.602188564242696e-09, 0.0, 0.000212743645752...",1090799


In [14]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [15]:
# Training graph (Modified)

train['id'] = train.index

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())
train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

test['id'] = test.index

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [16]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [17]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [18]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [19]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [20]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [21]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [22]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [23]:
# Convert to GPU
train_g = train_g.to('cuda')

In [ ]:
dgi.load_state_dict(torch.load('best_dgi_UNSW_v3.pkl'))

<All keys matched successfully>

In [26]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [27]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [28]:
# Convert to GPU
test_g = test_g.to('cuda')

In [29]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [30]:
# Multimodal (Fusion) Learning

df_train = pd.DataFrame(training_emb,)
# map the id to the original data
df_train['id'] = train_g.edata['id'].detach().cpu().numpy()


df_raw_train = pd.DataFrame(X_train.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_fuse_train = pd.merge(df_train, df_raw_train, on='id', how='left')
df_fuse_train = df_fuse_train.drop(columns=["id"])
df_fuse_train["Attacks"] = train_g.edata['Attack'].detach().cpu().numpy()
df_fuse_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb,)
# map the id to the original data
df_test['id'] = test_g.edata['id'].detach().cpu().numpy()

df_raw_test = pd.DataFrame(X_test.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_raw_test = pd.merge(df_test, df_raw_test, on='id', how='left')
df_fuse_test = df_raw_test.drop(columns=["id"])
df_fuse_test["Attacks"] = test_g.edata['Attack'].detach().cpu().numpy()
df_fuse_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

# Embeddings CBLOF  Embeddings

In [31]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [32]:
benign_fuse_train_samples = df_fuse_train[df_fuse_train.Label == 0].drop(columns=["Label", "Attacks"])
normal_fuse_train_samples = df_fuse_train.drop(columns=["Label", "Attacks"])

fuse_train_labels = df_fuse_train["Label"]
fuse_test_labels = df_fuse_test["Label"]

fuse_test_samples = df_fuse_test.drop(columns=["Label", "Attacks"])

In [33]:
fuse_test_samples

,0,1,2,3,4,5,6,7,8,9,...,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE
0,0.120862,0.009952,0.089464,0.078869,0.037023,0.041781,0.239061,0.191186,0.174297,0.032827,...,0.0,0.000000,0.542037,0.421585,8.162408e-08,8.515842e-08,9.755624e-07,9.757144e-07,0.000000,9.409448e-07
1,0.120862,0.009952,0.089464,0.078869,0.037023,0.041781,0.239061,0.191186,0.174297,0.032827,...,0.0,0.000039,0.025947,0.051894,2.380440e-07,2.548007e-07,7.641710e-08,7.642900e-08,0.000000,7.370546e-08
2,0.120862,0.009952,0.089464,0.078869,0.037023,0.041781,0.239061,0.191186,0.174297,0.032827,...,0.0,0.000000,0.000000,0.000000,1.134216e-08,1.134183e-08,4.866368e-08,4.713307e-10,0.000071,5.372992e-08
3,0.120862,0.009952,0.089464,0.078869,0.037023,0.041781,0.239061,0.191186,0.174297,0.032827,...,0.0,0.000056,0.408769,0.408769,4.573555e-06,4.834951e-06,1.324268e-06,1.324475e-06,0.000000,1.277277e-06
4,0.120862,0.009952,0.089464,0.078869,0.037023,0.041781,0.239061,0.191186,0.174297,0.032827,...,0.0,0.000000,0.000000,0.000000,1.037189e-08,1.037159e-08,4.450070e-08,4.310103e-10,0.000065,4.913355e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134570,-0.036482,-0.129115,0.111613,-0.015759,0.092214,0.212261,0.170611,0.119314,0.041390,0.022624,...,0.0,0.000000,0.000000,0.000000,5.627534e-08,5.627369e-08,2.763944e-07,2.764375e-07,0.000000,2.665866e-07
134571,-0.035830,-0.133259,0.113132,-0.015819,0.090947,0.212786,0.170340,0.118551,0.041846,0.021603,...,0.0,0.000000,0.000000,0.000000,2.390429e-07,1.521110e-06,3.188712e-07,3.189209e-07,0.000000,3.075562e-07
134572,-0.035830,-0.133259,0.113132,-0.015819,0.090947,0.212786,0.170340,0.118551,0.041846,0.021603,...,0.0,0.000000,0.000000,0.000000,2.106824e-07,1.340643e-06,2.810397e-07,2.810835e-07,0.000000,2.710671e-07
134573,-0.035830,-0.133259,0.113132,-0.015819,0.090947,0.212786,0.170340,0.118551,0.041846,0.021603,...,0.0,0.000000,0.000000,0.000000,2.125036e-07,1.352232e-06,2.834692e-07,2.835134e-07,0.000000,2.734104e-07


In [36]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [37]:
from pyod.models.cblof import CBLOF
n_est = [5,6,7,9,10] # can't be lower than 5, as it will not work with the dataset
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 30/30 [01:44<00:00,  3.47s/it]

{'n_estimators': 6, 'con': 0.01}
0.9435783813568277
              precision    recall  f1-score   support

           0     1.0000    0.9897    0.9948    129059
           1     0.8056    1.0000    0.8923      5516

    accuracy                         0.9901    134575
   macro avg     0.9028    0.9948    0.9436    134575
weighted avg     0.9920    0.9901    0.9906    134575



In [36]:
n_est = [5,6,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 30/30 [01:43<00:00,  3.45s/it]

{'n_estimators': 9, 'con': 0.04}
0.9733618820572987
              precision    recall  f1-score   support

           0     0.9980    0.9976    0.9978    129059
           1     0.9448    0.9530    0.9489      5516

    accuracy                         0.9958    134575
   macro avg     0.9714    0.9753    0.9734    134575
weighted avg     0.9958    0.9958    0.9958    134575



In [37]:
# HBOS  Embeddings+Raw (Multimodal)

In [38]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [39]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [01:30<00:00,  2.52s/it]

{'n_estimators': 5, 'con': 0.001}
0.9920366522431351
              precision    recall  f1-score   support

           0     1.0000    0.9987    0.9993    129059
           1     0.9699    1.0000    0.9847      5516

    accuracy                         0.9987    134575
   macro avg     0.9850    0.9993    0.9920    134575
weighted avg     0.9988    0.9987    0.9987    134575



In [40]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [01:33<00:00,  2.59s/it]

{'n_estimators': 5, 'con': 0.04}
0.9888236704486386
              precision    recall  f1-score   support

           0     0.9992    0.9990    0.9991    129059
           1     0.9764    0.9808    0.9786      5516

    accuracy                         0.9982    134575
   macro avg     0.9878    0.9899    0.9888    134575
weighted avg     0.9982    0.9982    0.9982    134575



In [41]:
##  PCA  Emb+Raw (Multimodal/Fusion) Learning

In [42]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [01:26<00:00,  2.41s/it]

{'n_estimators': 10, 'con': 0.001}
0.9915781501002836
              precision    recall  f1-score   support

           0     1.0000    0.9986    0.9993    129059
           1     0.9682    1.0000    0.9839      5516

    accuracy                         0.9987    134575
   macro avg     0.9841    0.9993    0.9916    134575
weighted avg     0.9987    0.9987    0.9987    134575



In [43]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [01:26<00:00,  2.41s/it]

{'n_estimators': 5, 'con': 0.04}
0.9864069483890199
              precision    recall  f1-score   support

           0     0.9988    0.9990    0.9989    129059
           1     0.9761    0.9717    0.9739      5516

    accuracy                         0.9979    134575
   macro avg     0.9875    0.9854    0.9864    134575
weighted avg     0.9979    0.9979    0.9979    134575



In [44]:
##  IF  Emb+Raw

In [45]:
for i in range(len(benign_fuse_train_samples.columns)):
    benign_fuse_train_samples.rename(columns={benign_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(normal_fuse_train_samples.columns)):
    normal_fuse_train_samples.rename(columns={normal_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(fuse_test_samples.columns)):
    fuse_test_samples.rename(columns={fuse_test_samples.columns[i]: f"feature {i}"}, inplace=True)

In [46]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples.to_numpy())
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/24 [00:00<?, ?it/s]/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  4%|▍         | 1/24 [00:00<00:16,  1.40it/s]/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  8%|▊         | 2/24 [00:01<00:15,  1.44it/s]/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 12%|█▎        | 3/24 [00:02<00:14,  1.44it/s]/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 17%

{'n_estimators': 20, 'con': 0.001}
0.9929099655002167
              precision    recall  f1-score   support

           0     1.0000    0.9988    0.9994    129059
           1     0.9732    1.0000    0.9864      5516

    accuracy                         0.9989    134575
   macro avg     0.9866    0.9994    0.9929    134575
weighted avg     0.9989    0.9989    0.9989    134575



In [47]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:23<00:00,  1.01it/s]

{'n_estimators': 20, 'con': 0.04}
0.9880084973777987
              precision    recall  f1-score   support

           0     0.9991    0.9990    0.9990    129059
           1     0.9759    0.9781    0.9770      5516

    accuracy                         0.9981    134575
   macro avg     0.9875    0.9885    0.9880    134575
weighted avg     0.9981    0.9981    0.9981    134575

